# Time Dependent Transformations

In this tutorial we discuss **time dependent transformations**. To see transformations between static datums, view the *datum transformation* tutorial.

For more details on time dependent transformations, refer to the [GDA2020 Technical Manual](https://www.anzlic.gov.au/sites/default/files/files/GDA2020%20Technical%20Manual%20V1.8_published.pdf).

Time dependent transformations are more complex than transformations between static datums. First we will complete a simple example.

## Common Example
Here we convert between **ATRF2014** and **GDA2020** (dynamic → static).
We will transform a coordinate in ATRF2014 at epoch **1/1/2011**.

First import GeodePy

In [ ]:
import geodepy.transform
from datetime import date

Use the function to transform from ATRF2014 to GDA2020.

In [ ]:
x_20, y_20, z_20, vcv = geodepy.transform.transform_atrf2014_to_gda2020(-4050762.770917, 4220880.800229, -2533400.199554, date(2011,1,1))
print(x_20, y_20, z_20)

This is the **GDA2020** coordinate.

## Transforming between Dynamic and Static Datums
The simplest time dependent transformations go between static and dynamic datums. This is because direct transformation parameters are often available.

Here we transform from **GDA94** to **ITRF2008** at **1/1/2007** (without a dedicated function).

In [ ]:
import geodepy.transform
import geodepy.constants
from datetime import date

To reach ITRF2008, we will use the **GDA94 → ITRF2008** transformation.

In [ ]:
x, y, z, vcv = geodepy.transform.conform14(-4050763.124034, 4220880.753100, -2533399.713463, date(2007,1,1), geodepy.constants.gda94_to_itrf2008)
print(x, y, z)

This is the **ITRF2008** coordinate on **1/1/2007**.

## Transforming Between Two Dynamic Datums
Transforming between two dynamic datums is more complex and requires extra steps and considerations.

Example: Transform **ITRF2008 (1/1/2007)** → **ITRF2020 (1/1/2030)**.

In [ ]:
import geodepy.constants
import geodepy.transform
from datetime import date

First convert ITRF2008 to **ITRF2014** (so plate motion can be applied). Plate motion can only be applied to **ITRF2014** or **ATRF2014**. Enter the **ITRF2008 epoch** so the result remains at that epoch.

In [ ]:
x, y, z, vcv = geodepy.transform.conform14(-4050762.612530, 4220880.821783, -2533400.416214, date(2007, 1, 1), geodepy.constants.itrf2008_to_itrf2014)
print(x, y, z)

Now we have the ITRF2014 coordinate at 1/1/2007.
Now move to **1/1/2030** using the **ITRF2014 → GDA2020** transformation which approximates plate motion in Australia. For other plates, use an appropriate plate motion model.

This process has to be done carefully. From 2007 to 2030 is **23 years** of motion. The reference epoch of the ITRF2014→GDA2020 transformation is **2020**. Subtract **23** from **2020** to get the desired epoch (enter **1/1/1997**).

> **Caution:** The plate motion model for Australia (itrf2014_to_gda2020) should only be used between 2005 - 2035. For transformations outside this range refer to the next section on older datum transformations. 

In [ ]:
x, y, z, vcv = geodepy.transform.conform14(x, y, z, date(1997, 1, 1), geodepy.constants.itrf2014_to_gda2020)
print(x, y, z)

This is now the **ITRF2014** coordinate at **1/1/2030**. Convert this to **ITRF2020** at **1/1/2030**.

In [ ]:
x, y, z, vcv = geodepy.transform.conform14(x, y, z, date(2030,1,1), geodepy.constants.itrf2014_to_itrf2020)
print(x, y, z)

This is the final coordinate in **ITRF2020** at **1/1/2030**.

## Transforming Between Older Dynamic Datums
The Australian plate motion model should only be used between **2005–2035**. For older datums, use a different method.

Example: Transform **ITRF88 (1/1/1988)** → **ITRF2014 (1/1/2030)**.

> **Caution:** This method only works for coordinates within Australia.

In [ ]:
import geodepy.constants
import geodepy.transform
from datetime import date

To go from an old dynamic datum to a more current datum, transformations to static datums should be completed first. In this case transforming to GDA2020 will give the most accurate result, before then transforming back to ITRF2014. To get to GDA2020 the ITRF88 coordinate first needs to be transformed to ITRF2014 at the 1/1/1988.

Transform ITRF88 to **ITRF2014** at **1/1/1988**.

In [ ]:
x, y, z, vcv = geodepy.transform.conform14(-4050763.124645, 4220880.752269, -2533399.717044, date(1988,1,1), geodepy.constants.itrf88_to_itrf2014)
print(x, y, z)

Now we can transform this to **GDA2020** (still entering **1988** epoch).

In [ ]:
x, y, z, vcv = geodepy.transform.conform14(x, y, z, date(1988,1,1), geodepy.constants.itrf2014_to_gda2020)
print(x, y, z)

Now change to **ITRF2014** at **1/1/2030**.

In [ ]:
x, y, z, vcv = geodepy.transform.conform14(x, y, z, date(2030,1,1), geodepy.constants.gda2020_to_itrf2014)
print(x, y, z)

This is now the **ITRF2014** coordinate at **1/1/2030**.